In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [28]:
data = pd.read_csv('vertebral_2C.csv')

In [29]:
data.head()

,pelvic incidence,pelvic tilt,lumbar lordosis angle,sacral slope,pelvic radius,grade of spondylolisthesis,Class Label
0,63.03,22.55,39.61,40.48,98.67,-0.25,AB
1,39.06,10.06,25.02,29.00,114.41,4.56,AB
2,68.83,22.22,50.09,46.61,105.99,-3.53,AB
3,69.30,24.65,44.31,44.64,101.87,11.21,AB
4,49.71,9.65,28.32,40.06,108.17,7.92,AB


data[0][7] = "Flow1"
data[0][8] = "Flow2"
data[0][10] = "Nitrogen1"
data[0][11] = "Nitogen2"
data[0][13] = "Fre1"
data[0][14] = "Fre2"

In [30]:
#data1= pd.DataFrame(data[1:,:],columns=data[0])

In [31]:
X = data.drop(['Class Label'],axis=1)


In [32]:
T = data['Class Label'].apply(lambda x: 1 if x == 'AB' else -1)

In [33]:
a = np.ones(shape=(len(X),1))
a = pd.DataFrame(a,columns=['bias'])
Xa = pd.concat([a,X],axis=1)
Xamatrix = Xa.as_matrix()
Xamatrix

array([[   1.  ,   63.03,   22.55, ...,   40.48,   98.67,   -0.25],
       [   1.  ,   39.06,   10.06, ...,   29.  ,  114.41,    4.56],
       [   1.  ,   68.83,   22.22, ...,   46.61,  105.99,   -3.53],
       ..., 
       [   1.  ,   61.45,   22.69, ...,   38.75,  125.67,   -2.71],
       [   1.  ,   45.25,    8.69, ...,   36.56,  118.55,    0.21],
       [   1.  ,   33.84,    5.07, ...,   28.77,  123.95,   -0.2 ]])

In [34]:
Xpinv = np.linalg.pinv(Xamatrix)

In [35]:
W = Xpinv.dot(T)
W

array([ 3.28627999, -3.09249233,  3.10369415,  0.00703623,  3.07617879,
       -0.02610257,  0.01081927])

In [36]:
trainOutput = Xa.dot(W)
trainOutput
predtrainOutputbinary = np.ones(trainOutput.shape[0])
for i in range(len(trainOutput)):
    if trainOutput[i] < 0:
        predtrainOutputbinary[i] = -1
predtrainOutputbinary
TP=0
TN=0
FP=0
FN=0
for i in range(len(predtrainOutputbinary)):
    if (T[i] == 1 and predtrainOutputbinary[i] == 1):
        TP = TP +1
    elif(T[i] == -1 and predtrainOutputbinary[i] == 1):
        FP = FP + 1
    elif (T[i] == -1 and predtrainOutputbinary[i] == -1):
        TN = TN +1
    else:
        FN = FN +1
print("TP " , TP)
print("TN " ,TN)
print("FP ", FP)
print("FN ", FN)

TP  196
TN  68
FP  32
FN  14


In [37]:
accuracybinary = (TP + TN) / (TP + TN + FP + FN)
accuracybinary

0.8516129032258064

In [38]:
sensitivity = TP / (TP + FN)
sensitivity

0.9333333333333333

In [39]:
specificity = TN / ( TN + FP)
specificity

0.68

In [40]:
PPV = TP / (TP + FP)
PPV

0.8596491228070176

In [223]:
a = np.ones(shape=(50,1))
xatest = np.concatenate((a,test_data[1:,:]),axis=1)

In [224]:
test_data.shape

(51, 15)

In [225]:
outY = xatest.dot(W)
outY

array([-0.23738757028648724, -1.1004610735880167, 1.0809319627350167,
       -1.129995667451246, -0.9424211835495389, -0.4203467629520403,
       1.194618389545151, -0.9378304594229214, -0.32587125845251885,
       -0.27254044885017625, 0.9318372316106751, -0.282205427425007,
       -0.21172900925063567, -1.039494650956153, 1.2290151058958199,
       -0.23460155029107452, -0.4591618430151547, -0.33646129053701923,
       -0.27966891219696444, 1.1006326588363866, -0.3520822187080215,
       -0.35023289700930493, -0.9797649271164327, -0.39890809506870895,
       -0.9966440738810657, -0.27467158238132017, 0.9638537468444631,
       -1.0145624260065678, -1.0019897545804803, -1.0690086559042615,
       0.27488969955196296, 1.1776605189951312, 0.24408002029287493,
       1.0009467543507875, -0.3311819497576264, -0.2083010067837812,
       -0.15317365185770507, -0.4015257343342382, 0.8374456805335064,
       -0.3815962236561301, 1.0304151095695064, -1.0271350974326552,
       -0.1255219922063

In [318]:
predict = np.ones(shape=outY.shape)

In [320]:
for i in range(len(outY)):
    if outY[i] < 0:
        predict[i] = -1
predict
predict.shape

(50,)

In [67]:
###start multiclass classification 
datamul = pd.read_csv('vertebral_3C.csv')


In [69]:
Tmulcol[0]

'DH'

In [72]:
Tmulcol = datamul['Class Label']
def changeInputs(x):
    if x == 'DH':
        return 0
    elif x == 'NO':
        return 1
    else:
        return 2
Tmulcol = Tmulcol.apply(lambda x : changeInputs(x))
        
Tmul = pd.get_dummies(Tmulcol)


In [74]:
for i in Tmul.columns:
    Tmul[i] = Tmul[i].apply(lambda x : -1 if x == 0 else x)
Wmul = Xpinv.dot(Tmul)

In [77]:
Wmul

array([[  2.12204076e+00,  -3.28627999e+00,   1.64239232e-01],
       [  1.81131978e+00,   3.09249233e+00,  -4.90381211e+00],
       [ -1.79823559e+00,  -3.10369415e+00,   4.90192974e+00],
       [ -1.24784149e-02,  -7.03623133e-03,   1.95146462e-02],
       [ -1.83173387e+00,  -3.07617879e+00,   4.90791267e+00],
       [ -1.19944611e-02,   2.61025731e-02,  -1.41081120e-02],
       [ -9.38587110e-04,  -1.08192702e-02,   1.17578573e-02]])

In [78]:
X = datamul.drop(['Class Label'],axis=1)
a = np.ones(shape=(len(X),1))
a = pd.DataFrame(a,columns=['bias'])
Xa = pd.concat([a,X],axis=1)
Xamatrix = Xa.as_matrix()
Xamatrix

array([[   1.  ,   63.03,   22.55, ...,   40.48,   98.67,   -0.25],
       [   1.  ,   39.06,   10.06, ...,   29.  ,  114.41,    4.56],
       [   1.  ,   68.83,   22.22, ...,   46.61,  105.99,   -3.53],
       ..., 
       [   1.  ,   61.45,   22.69, ...,   38.75,  125.67,   -2.71],
       [   1.  ,   45.25,    8.69, ...,   36.56,  118.55,    0.21],
       [   1.  ,   33.84,    5.07, ...,   28.77,  123.95,   -0.2 ]])

In [79]:
trainOutputMult = Xa.dot(Wmul)
trainOutputMult
predtrainOutputmult = np.zeros(trainOutputMult.shape[0],dtype=int)
# predtrainOutputmult[i] = pd.a(trainOutputMult[i:i+1,:])
predtrainOutputmult= trainOutputMult.idxmax(axis=1)
predtrainOutputmult.shape
confMatrix = np.zeros(shape=(3,3),dtype=int)
for i in range(len(predtrainOutputmult)):
    row = Tmulcol[i]
    col = predtrainOutputmult[i]
    print("row = col = " , row ,col)
    confMatrix[row,col] = confMatrix[row,col] + 1;
    print("element is " , confMatrix[row,col])
confMatrix

row = col =  0 0
element is  1
row = col =  0 0
element is  2
row = col =  0 2
element is  1
row = col =  0 2
element is  2
row = col =  0 1
element is  1
row = col =  0 1
element is  2
row = col =  0 1
element is  3
row = col =  0 1
element is  4
row = col =  0 1
element is  5
row = col =  0 0
element is  3
row = col =  0 1
element is  6
row = col =  0 0
element is  4
row = col =  0 0
element is  5
row = col =  0 0
element is  6
row = col =  0 0
element is  7
row = col =  0 1
element is  7
row = col =  0 2
element is  3
row = col =  0 1
element is  8
row = col =  0 1
element is  9
row = col =  0 0
element is  8
row = col =  0 1
element is  10
row = col =  0 1
element is  11
row = col =  0 2
element is  4
row = col =  0 1
element is  12
row = col =  0 0
element is  9
row = col =  0 0
element is  10
row = col =  0 0
element is  11
row = col =  0 0
element is  12
row = col =  0 0
element is  13
row = col =  0 1
element is  13
row = col =  0 0
element is  14
row = col =  0 0
element is  1

array([[ 33,  22,   5],
       [  8,  81,  11],
       [  2,   8, 140]])

In [80]:
confMatrix

array([[ 33,  22,   5],
       [  8,  81,  11],
       [  2,   8, 140]])

In [82]:
ppv = np.zeros(shape= (3,1))
#calculate PPV for each class
for i in range(3):
    TPi = confMatrix[i][i]
    FPI = confMatrix[:,i:i+1].sum()
    ppv[i] = TPi/(FPI)
print("PPV max " , ppv.max())
print(" PPV min " , ppv.min() )
print(" PPV max class " , np.argmax(ppv))
print("PPv min class" , np.argmin(ppv) )

PPV max  0.897435897436
 PPV min  0.72972972973
 PPV max class  2
PPv min class 1
